## **실습 목표**
적절한 데이터 전처리와 로딩 방법은 모델의 성능과 학습 속도에 큰 영향을 미친다. PyTorch는 모델 학습에 필요한 데이터를 저장, 관리하는 객체인 **Dataset**, Dataset에서 배치 단위로 데이터를 로딩하여 학습에 효율적으로 사용하는 **DataLoader**를 제공한다. 이 도구들의 기본 원칙과 사용 방법을 이해하고, 실제 데이터셋에 적용해보자.

시간이 지날수록 어떻게 하면 대용량 데이터를 잘 넣어서 학습을 시키냐가 중요한 이슈가 됨.    
파이토치는 대용량 데이터를 잘 다룰 수 있는 dataset API를 제공하고 있음.

### **Dataset 클래스**
- 데이터 **입력 형태**를 정의하는 클래스
- 데이터를 입력하는 방식의 표준화
- 데이터 형태에 따라 각 함수를 다르게 정의
- \_\_init\_\_()
    - 초기화
- \_\_len\_\_()
    - 데이터의 전체 길이
- \_\_getitem\_\_()
    - index값이 주어졌을 때, 반환되는 데이터의 형태를 정의(X,y)
-모든 것을 데이터 생성 시점에 처리할 필요는 없음
    - \_\_init\_\_()에서 모든걸 처리하지 않아도 됨. 학습이 필요한 시점에 해 주자

**❓퀴즈**   
### **DataLoader 클래스**
- **🖊 정답:** Data의 Batch를 생성해주는 클래스
- 학습 직전(GPU feed 전)데이터의 변환(Transforms)을 책임
- Tensor로 변환, Batch 처리가 메인 업무
- 병렬적인 데이터 전처리 코드의 고민 필요
    - 전처리는 일반적으로 Dataset 클래스에서 수행되거나, 별도의 전처리 파이프라인을 통해 이루어진다.

In [ ]:
import os
import sys
import torch
import requests
import tarfile
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from skimage import io, transform
from torchvision import transforms, datasets
from torchvision.datasets import VisionDataset
from typing import Any, Callable, Dict, List, Optional, Tuple
from torch.utils.data import Dataset, DataLoader

### **1. Custom Dataset 복습하기**

In [ ]:
# 파이토치의 Dataset 클래스를 상속받아 CustomDataset 클래스를 정의
class CustomDataset(Dataset):
    def __init__(self, text, labels):
        self.labels = labels
        self.text = text

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx): # dataloader에서 호출하게 됨
        label = self.labels[idx]
        text = self.text[idx]
        sample={'Text': text, 'Class': label}

        return sample

In [ ]:
text = ['Happy', 'Amazing', 'Sad', 'Unhapy', 'Glum']
labels = ['Positive', 'Positive', 'Negative', 'Negative', 'Negative']

MyDataset = CustomDataset(text, labels)

In [ ]:
type(MyDataset)

__main__.CustomDataset

In [ ]:
MyDataLoader = DataLoader(MyDataset, batch_size=2, shuffle=True)

for dataset in MyDataLoader:
    print(dataset)

{'Text': ['Sad', 'Unhapy'], 'Class': ['Negative', 'Negative']}
{'Text': ['Amazing', 'Glum'], 'Class': ['Positive', 'Negative']}
{'Text': ['Happy'], 'Class': ['Positive']}


### **2. NotMNIST를 이용하여 Custom Dataset 만들기**

In [ ]:
# empty file이라고 나온다... 성공했다고 치고 듣자
class NotMNIST(VisionDataset):
    resource_url = 'http://yaroslavvb.com/upload/notMNIST/notMNIST_large.tar.gz'

    def __init__(self, root:str, train:bool = True,
                 # Optional[x] = x or None
                 # 기본값이 정해지지 않은(None이 허용되는) 파라미터에 사용하는 것이 적합
                 # Callable: 호출 가능한 객체를 의미.
                 #           1. __call__ 메서드가 있는 클래스 인스턴스거나,
                 #            2. 호출 가능한 메서드나 함수.
                 transform: Optional[Callable] = None,
                 target_transform: Optional[Callable] = None,
                 download: bool = False):
        super(NotMNIST, self).__init__(root, transform=transform, target_transform=target_transform)

        #_check_exists()는 Dataset에서 Object Detection data parsing 파일 존재 여부를 확인하는 함수
        if not self._check_exists():
            self.download()

        self.data, self.targets = self._load_data()

    # dataset의 전체 길이를 반환
    def __len__(self):
        return len(self.data)

    # load한 data를 차례차례 돌려줌
    def __getitem__(self, index):
        image_name = self.data[index]
        image = io.imread(image_name) # numpy array를 읽어온다
        label = self.targets[index]

        if self.transform: # 이렇게 굳이 안 하고 나중에 Transforms 객체 써도 된다.
            image = self.transform(image)

        return image, label

    def _load_data(self): # 데이터를 받아서 데이터 리스트를 만든다
        filepath = self.image_folder
        data = []
        targets = []

        # 각 클래스 별로 데이터 로드
        for target in os.listdir(filepath):
            # abspath: 절대경로 구하기
            filenames = [os.path.abspath(os.path.join(filepath, target, x)) for x in os.listdir(os.path.join(filepath, target))]
            targets.extend([target] * len(filenames))
            data.extend(filenames)
        # 파일 리스트와 target을 정의 해준다
        # 이미지 파일은 폴더에 저장되어 있고, 각각의 폴더가 label이 되기 때문
        return data, targets #filename = data, label = targets

    # @: decorator: 어떤 함수를 꾸며서 새 함수로 만드는 기능
    # property함수가 raw_folder 함수를 꾸며주고 있는 구조.
    # raw_folder = property(raw_folder)
    # 즉, raw_folder가 property의 result가 되는 것.
    # property를 붙이면 NotMNIST.raw_folder하면 raw_folder를 실행시킬 수 있음.
    # 원본 데이터 폴더 경로
    @property
    def raw_folder(self) -> str:
        return os.path.join(self.root, self.__class__.__name__, 'raw')

    # 이미지 폴더 경로
    @property
    def image_folder(self) -> str:
        return os.path.join(self.root, 'notMNIST_large')

    # 데이터 다운로드 함수
    def download(self) -> None:
        os.makedirs(self.raw_folder, exist_ok=True)
        os.makedirs(self.image_folder, exist_ok=True)
        fname = self.resource_url.split("/")[-1]
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6422.142 Safari/537.36'
        response = requests.head(self.resource_url, headers={"User-Agent": user_agent})
        filesize = int(response.headers.get("Content-Length", 0))  # Use 0 if not found
        # 데이터 다운로드 진행 상황 표시
        with requests.get(self.resource_url, stream=True, headers={"User-Agent": user_agent}) as r, \
                open(os.path.join(self.raw_folder, fname), "wb") as f, \
                tqdm(unit="B", unit_scale=True, unit_divisor=1024, total=filesize, file=sys.stdout, desc=fname) as progress:
            for chunk in r.iter_content(chunk_size=1024):
                datasize = f.write(chunk)
                progress.update(datasize)

            self._extract_file(os.path.join(self.raw_folder, fname), target_path=self.root)

    # 파일 압축 해제 함수
    def _extract_file(self, fname, target_path) -> None:
        tag = 'r:gz' if fname.endswith('tar.gz') else 'r:'
        with tarfile.open(fname,tag) as tar:
            tar.extractall(path=target_path)

    # 데이터셋 존재 여부 확인 함수
    def _check_exists(self) -> bool:
        return os.path.exists(self.raw_folder)

In [ ]:
# 데이터셋 생성
dataset = NotMNIST("data", download = True)

In [ ]:
fig = plt.figure()

# 8개의 샘플 이미지 출력
for i in range(8):
    sample = dataset[i]
    # 1행 4열의 서브플롯 중 i+1번째 위치에 그래프 그리기
    ax = plt.subplot(1, 4, i+1)
    # 레이아웃을 조절하여 그래프 간의 간격을 최적화
    plt.tight_layout()
    # 서브플롯의 제목 설정
    ax.set_title('Sample #{}'.format(i))
    # 서브플롯 축 숨기기
    ax.axis('off')
    plt.imshow(sample[0])

    # 4개의 샘플 이미지 출력 후, 그림을 화면에 표시하고 반복문을 종료.
    if i == 3:
        plt.show()
        break

In [ ]:
# 데이터 전처리를 위한 변환(Transform) 객체를 생성.
# 여러 전처리 단계를 순차적으로 적용하기 위해 Compose를 사용(PyTorch transforms.Compose)
data_transform = transforms.Compose([

        # 224X224 크기로 무작위로 이미지를 잘라냄.
        transforms.RandomCrop(224)
        # 0.5의 확률로 이미지를 수평으로 뒤집음.
        transforms.RandomHorizontalFlip(),
        # 이미지를 텐서(Tensor) 형태로 변환
        transforms.ToTensor(),
        # 주어진 평균과 표준편차를 사용하여 이미지를 정규화.
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
        ])

# NotMNIST 데이터셋 로드.(다운로드 X)
dataset = NotMNIST("data", download=False)

In [ ]:
# 데이터셋을 배치 크기로 나누어 로드하기 위한 DataLoader 객체 생성.
# 배치 크기 128, 데이터 셔플해서 로드
# iterable한 generator로 만들어 준다
dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
# DataLoader에서 첫 번째 배치의 특성(features)와 labels 가져오기.
# next는 반복자의 바로 다음 항목을 가져오는 함수.
train_features, train_labels = next(iter(dataset_loader))

In [ ]:
# 첫 번째 배치 특성의 형태(shape) 출력
# 128,28,28
train_features.shape

In [ ]:
# 첫 번째 배치 레이블 출력
train_labels